In [ ]:
import requests
import csv
import json


# Tạo một phiên làm việc
s = requests.Session()

# Định nghĩa URL của API
api_base = "https://api.endpoints.anyscale.com/v1"
token = "esecret_cmhwxyie53d1xj4jnff64phvwx"
url = f"{api_base}/chat/completions"

# Hàm gửi yêu cầu đến API
def query(title):
    body = {
        "model": "meta-llama/Meta-Llama-3-70B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": f"You are a professional analyst finding the information related to medical field. Answer whether the title are related to the medical field: '{title}'? Answer with 'yes' or 'no'."
            }
        ],
        "temperature": 0.5,
        "max_tokens": 50,
        "top_p": 1,
        "frequency_penalty": 0
    }
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = s.post(url, headers=headers, data=json.dumps(body))
    return response.json()

# Hàm kiểm tra tiêu đề có liên quan đến COVID-19 hay không
def is_medical_related(title):
    response = query(title)
    if response and 'choices' in response:
        answer = response['choices'][0]['message']['content'].strip().lower()
        return 1 if 'yes' in answer else 0
    return 0

# Đọc dữ liệu từ file CSV
input_file = 'C:\\Users\\DS 10 DIEM\\Downloads\\video_data.csv'
output_file = 'C:\\Users\\DS 10 DIEM\\Downloads\\final_data.csv'

titles= []
with open(input_file, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        titles.append((row['title']))

# Phân tích các tiêu đề
results = []
for title in titles:
    is_related = is_medical_related(title)
    results.append({
        "title": title,
        "medical_field": is_related
    })

# Lưu kết quả vào file CSV
with open(output_file, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["title", "medical_field"])
    writer.writeheader()
    for result in results:
        writer.writerow(result)

print(f"Kết quả đã được lưu vào {output_file}")
